## TIANGE YU (Jill)
## Inspection intro

- What data structure did you use to hold your sequence motifs (DNA)?         
I am planning to use list of string to hold motifs 

- Is your profile  structure based on counts or frequencies?            
I think it is more easier to biuld profile with frequencies     

- Where did you implement iterations (mostly why did you choose this)?          
The iteration should be after each run of random motif search. Cause in that case you can compare each best score of random motif search to aviod program trapped in local maximum.   

## Inspection Result    

- In terms of profile: we think it is more intuitive to use dictionary to store profile with keys of ATGC. I was initially thinking of storing profile using 2d list, but it turns out that 2d list makes it harder to access the keys and it is not memory efficient.        

- I was intially misunderstund the concept of pseudocounts. I thought pseudocounts were only used when certain base has zero probablity, that causes deviation of the profile. Then I realized that pseudocounts is added to all base counts.

- It is more useful when profile is in frequencies beacause that makes the calculate of score easier

In [49]:
class CommandLine():
    '''
    Handle the command line, usage and help requests.

    CommandLine uses argparse, now standard in 2.7 and beyond. 
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.

    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.

    '''

    def __init__(self, inOpts=None):
        '''
        CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        '''

        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
        )

        self.parser.add_argument('-i', type=int, required=True)
        self.parser.add_argument('-k', type=int, required=True)
        self.parser.add_argument('-p', type=float, required=True)

        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')
        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)


In [50]:
class Motif():
    '''
    generate profiles with given motif, with pseudocounts
    input: 
        Dna a list of dna sequence
        pc is integer of pseudocounts
        ms is list of strings of initiated motifs
    output:
        dictionary with A, C, G, T as key
    ''' 
    def profile(self, Dna, pc, ms):
        n = 4*pc + len(Dna)
        p = {'a':[], 'c':[], 'g':[], 't':[]}
        for i in range(0, len(ms[0])):
            temp = [n[i] for n in ms]
            p['a'].append((temp.count('a')+pc)/n)
            p['c'].append((temp.count('c')+pc)/n)
            p['g'].append((temp.count('g')+pc)/n)
            p['t'].append((temp.count('t')+pc)/n)
            temp.clear()    
        return p
    
    '''
    get new motif based on given profile
    input:
        Dna: a list of string of dna sequence
        pr: profile in dictionary form
        k: integer represent k-mer
    output:
        motif: list of string
    '''
    def motif(self, Dna, pr, k):
        
        from math import prod

        motifs = []
        for seq in Dna:
            ss = []
            for x in range(0, len(seq)-k+1):
                ss.append(seq[x:x+k])

            # calculate probability of each possible motif
            prob_temp = []
            for i in range(0, len(ss)):
                list = []
                for j in range(0, k):
                    list.append(pr[ss[i][j]][j])
                prob_temp.append(prod(list))
                
            # find motif with max possibility
            max_v = max(prob_temp)
            max_i = prob_temp.index(max_v)
            better_m = ss[max_i]
            motifs.append(better_m) 
        return motifs
    
    '''
    use relative entropy method to get relative distance with null model
    input: 
        pr: dictionary form of profile
        Dna: a list of string of sequence
        pc: int of ps count
        k: int of k-mer length
    output:
        float of score
    '''
    def getScore(self, pr, Dna, pc, k):
        
        from math import log2
        
        # construct null model  
        seq = ''
        for s in Dna:
            seq += s 
        null = {} # Q(i)
        null['a'] = (seq.count('a')+pc)/(len(seq)+4*pc)
        null['c'] = (seq.count('c')+pc)/(len(seq)+4*pc)
        null['g'] = (seq.count('g')+pc)/(len(seq)+4*pc)
        null['t'] = (seq.count('t')+pc)/(len(seq)+4*pc)
        
        # experimental model
        score = 0
        for i in range(0, k):
            for key in list(pr.keys()):            
                score += (pr[key][i])*log2(pr[key][i]/null[key])
        
        return score
    
    def RandomizedMotifSearch(self, Dna, k, pc):
        
        from random import randint

        # initiation: randomly pick motif
        motifs = []
        for s in Dna:
            start = randint(0, len(s)-k)
            motifs.append(s[start:start+k])
        
        bestS = 0
        bestM = motifs
        while True:
            cur_pr = self.profile(Dna, pc, motifs)
            motifs = self.motif(Dna, cur_pr, k)
            cur_score = self.getScore(self.profile(Dna, pc, motifs), Dna, pc, k)
            if cur_score > bestS:
                bestS = cur_score
                bestM = motifs
            else:
                return bestM

In [51]:
def main (inFile=None, options = None):
    
    # initialize command line object
    cl = CommandLine(options)
    
    # read fastA file to get DNA
    f = open(inFile, 'r')
    dna = []
    for r in f.readlines():
        if '>' not in r:
            dna.append(r.strip())  
            
    # initialize motif object
    motif = Motif()
    
    # repeat
    BM = []
    bestScore = 0
    for r in range(cl.args.i):
        curMotif = motif.RandomizedMotifSearch(dna, cl.args.k, cl.args.p)
        curProfile = motif.profile(dna, cl.args.p, curMotif)
        curScore = motif.getScore(curProfile, dna, cl.args.p, cl.args.k)
        if curScore > bestScore:
            bestScore = curScore
            BM = curMotif

    # find consensus
    t_p = motif.profile(dna, cl.args.p, BM)
    consensus = ''
    for i in range(0, cl.args.k):
        temp = []
        for k in list(t_p.keys()):
            temp.append(t_p[k][i])
        max_v = max(temp)
        max_i = temp.index(max_v)
        if max_i == 0:
            consensus += 'A'
        elif max_i == 1:
            consensus += 'C'
        elif max_i == 2:
            consensus += 'G'
        else:
            consensus += 'T'       
    
    print(inFile + ': ')
    print(bestScore)
    print(consensus + '\n')   
    
       
if __name__ == "__main__":
    main(inFile='p1860Crisprs', options=['-i 1000', '-k 13', '-p 1'])
    main(inFile='paeCrisprs', options=['-i 5000', '-k 13', '-p 1'])
    main(inFile='parCrisprs', options=['-i 1000', '-k 13', '-p 1'])
    main(inFile='pcaCrisprs', options=['-i 1000', '-k 13', '-p 1'])
    main(inFile='pisCrisprs', options=['-i 1000', '-k 13', '-p 1'])
    main(inFile='pogCrisprs', options=['-i 1000', '-k 13', '-p 1'])
    main(inFile='sum.fa', options=['-i 1000', '-k 13', '-p 1'])

p1860Crisprs: 
8.657886340591507
CGAAAAACTTAAA

paeCrisprs: 
5.402943872451918
CAAAGAAAAACTT

parCrisprs: 
7.9779836447711565
CCAAGAAAAACTT

pcaCrisprs: 
7.783555496977209
CAAAGAAAAACTT

pisCrisprs: 
8.177546321992208
CCCAAAAGATCCA

pogCrisprs: 
7.550960478104058
CCAAAAGAACTGT

sum.fa: 
11.890713001365832
GAAAAACTTAAAA

